In [5]:
import numpy as np
import random
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from qiskit.quantum_info import DensityMatrix, random_density_matrix
from qiskit.quantum_info.operators import Operator
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.linalg import sqrtm
from scipy.optimize import minimize


# 1) 2 Qubit Analysis

## 1.1) Load The Datasets

In [ ]:
freq_2q = np.load('../data/2_qubit_ma/noise/num100000_cube_9_N=100000_noise0.0100.npy')
print(f'2 Qubit Frequency Data Shape: {freq_2q.shape}')

rho_2 = np.load('../data/2_qubit_ma/level4num100000_pure_haar.npy')
print(rho_2.shape)
print(rho_2)

(100000, 36)
(100000, 4, 4)
[[[ 0.21499355+0.j         -0.30373088-0.0047481j
   -0.14253069-0.02100995j -0.14300953-0.18785141j]
  [-0.30373088+0.0047481j   0.42919896+0.j
    0.20182343+0.02653391j  0.20618457+0.26222767j]
  [-0.14253069+0.02100995j  0.20182343-0.02653391j
    0.0965444 +0.j          0.11316618+0.11056132j]
  [-0.14300953+0.18785141j  0.20618457-0.26222767j
    0.11316618-0.11056132j  0.25926308+0.j        ]]

 [[ 0.20897381+0.j         -0.3062431 -0.14170762j
    0.06937363-0.162879j   -0.03196807-0.13810725j]
  [-0.3062431 +0.14170762j  0.54488121+0.j
    0.00878579+0.28573603j  0.14050016+0.18071297j]
  [ 0.06937363+0.162879j    0.00878579-0.28573603j
    0.14998184+0.j          0.09703145-0.07076451j]
  [-0.03196807+0.13810725j  0.14050016-0.18071297j
    0.09703145+0.07076451j  0.09616314+0.j        ]]

 [[ 0.21460249+0.j         -0.187629  -0.2088956j
   -0.05586883-0.01352221j  0.14962502+0.25301065j]
  [-0.187629  +0.2088956j   0.36738636+0.j
    0.06200927-0

## 1.2) Analyse the Structure of the Dataset

Number of samples: $N = 100{,}000$.

Density matrices: each sample $\rho^{(k)}\in\mathbb{C}^{4\times4}$ satisfies 
$$
\rho^{(k)\dagger} = \rho^{(k)},\quad \mathrm{Tr}\,\rho^{(k)} = 1.
$$
For the pure‐Haar file also 
$$
\rho^{(k)\,2} = \rho^{(k)}.
$$

Hilbert‐space and measurement dimensions:
$$
d = 2^2 = 4,\quad K = 3^2 = 9,\quad d \cdot K = 36.
$$

Measurement projectors: a set $\{M_i\}_{i=1}^{36}$ of Pauli‐product projectors.

Frequency vectors: for each $\rho^{(k)}$ the true probabilities are
$$
p_i^{(k)} = \mathrm{Tr}\bigl(\rho^{(k)}\,M_i\bigr),
$$
and with $S$ copies the empirical frequencies are
$$
f_i^{(k)} = \frac{1}{S}\,\mathrm{Binomial}\bigl(S,\,p_i^{(k)}\bigr),
\quad
f^{(k)}=(f_1^{(k)},\dots,f_{36}^{(k)})\in[0,1]^{36}.
$$

Once we process the data, we will eventually perform a Cholesky Decomposition on each of the density matrices $\rho^{(k)}$ such that the resulting recomposed matrix is a lower triangular matrix.

Cholesky parameters: each $\rho^{(k)}$ has a lower‐triangular factor $\rho_L^{(k)}$ with
$$
\rho^{(k)} = \frac{\rho_L^{(k)}\,\rho_L^{(k)\dagger}}{\mathrm{Tr}(\rho_L^{(k)}\,\rho_L^{(k)\dagger})},
\qquad
\alpha^{(k)} = \operatorname{vec}\bigl(\rho_L^{(k)}\bigr)\in\mathbb{R}^{16}.
$$




# 2) Baseline Implementations

## 2.1) Maximum Likelihood Estimation

This approach can be characterised by:

-  Compute $\alpha^* \: = \argmax\{\; \mathcal{L}(\alpha, \mathbf{X}) \;\}$
  
   Here $\alpha$ is the flattened cholesky decomp, so we switch between $\rho$ and $\alpha$ within the calculation such that the optimisation returns a Cholesky matrix

- Reconstruct $\alpha^* -> \; \rho^*$

In [16]:
# quick split to improve runtimes

X_train, X_test, y_train, y_test = train_test_split(freq_2q, rho_2, train_size=0.1, random_state=2)

In [17]:
# We calculate the maximum Likelihood using the rho's
X = X_train
y = y_train
N = X.shape[0]

#Restore the counts from the frequencies
shots = 1024
counts = (X * shots).astype(int)  # shape (N, 36)

# Build computational basis projectors for 2 qubits
proj = []
for m in range(4):
    P = np.zeros((4,4), dtype=complex)
    P[m, m] = 1
    proj.append(P)

# Define basis-change unitaries for X, Y, Z on one qubit
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
Sdg = np.array([[1, 0], [0, -1j]])
bases = {
    'X': H,
    'Y': Sdg @ H,
    'Z': np.eye(2)
}

# Build the POVM elements E_jm using the 9 joint Pauli unitaries
settings = []
for b1 in ['X','Y','Z']:
    for b2 in ['X','Y','Z']:
        U = np.kron(bases[b1], bases[b2])
        settings.append(U)

E = []
for U in settings:
    U_dag = U.conj().T
    for P in proj:
        E.append(U_dag @ P @ U)
# E contains the 36 POVM elements

# Map the cholesky parameterised vector to a PSD trace 1 density matrix
def alpha_to_rho(alpha):
    # alpha: length 16
    L = np.zeros((4,4), dtype=complex)
    idx = 0
    # diagonal entries (real, positive)
    for i in range(4):
        L[i, i] = alpha[idx]
        idx += 1
    # lower-triangular off-diagonals (real + imag)
    for i in range(1, 4):
        for j in range(i):
            re = alpha[idx]; im = alpha[idx+1]
            L[i, j] = re + 1j * im
            idx += 2
    rho = L @ L.conj().T
    return rho / np.trace(rho)

# COmpute NLL
def neg_log_likelihood(alpha, count):
    rho = alpha_to_rho(alpha)
    # avoid log(0) by clipping
    probs = np.array([np.real(np.trace(Ej @ rho)) for Ej in E])
    probs = np.clip(probs, 1e-12, 1.0)
    return -np.sum(count * np.log(probs))

#initialise the estimator
rho_est = np.zeros((N, 4, 4), dtype=complex)

for i in range(N):
    
    # initial guess: uniform identity
    init = np.zeros(16)
    init[:4] = np.sqrt(1/4)  # diagonal entries ~ sqrt(1/4)
    
    res = minimize(
        neg_log_likelihood, init, args=(counts[i],),
        method='L-BFGS-B',
        options={'maxiter': 500}
    )
    print(f"COmpleted iter {i}")
    #reconstruct the density matrix from cholesky decomp
    rho_est[i] = alpha_to_rho(res.x)



COmpleted iter 0
COmpleted iter 1
COmpleted iter 2
COmpleted iter 3
COmpleted iter 4
COmpleted iter 5
COmpleted iter 6
COmpleted iter 7
COmpleted iter 8
COmpleted iter 9
COmpleted iter 10
COmpleted iter 11
COmpleted iter 12
COmpleted iter 13
COmpleted iter 14
COmpleted iter 15
COmpleted iter 16
COmpleted iter 17
COmpleted iter 18
COmpleted iter 19
COmpleted iter 20
COmpleted iter 21
COmpleted iter 22
COmpleted iter 23
COmpleted iter 24
COmpleted iter 25
COmpleted iter 26
COmpleted iter 27
COmpleted iter 28
COmpleted iter 29
COmpleted iter 30
COmpleted iter 31
COmpleted iter 32
COmpleted iter 33
COmpleted iter 34
COmpleted iter 35
COmpleted iter 36
COmpleted iter 37
COmpleted iter 38
COmpleted iter 39
COmpleted iter 40
COmpleted iter 41
COmpleted iter 42
COmpleted iter 43
COmpleted iter 44
COmpleted iter 45
COmpleted iter 46
COmpleted iter 47
COmpleted iter 48
COmpleted iter 49
COmpleted iter 50
COmpleted iter 51
COmpleted iter 52
COmpleted iter 53
COmpleted iter 54
COmpleted iter 55
CO

In [ ]:
# compute Fidelity
def fidelity(rho1, rho2):
    sqrt_rho1 = sqrtm(rho1)
    F = np.trace(sqrtm(sqrt_rho1 @ rho2 @ sqrt_rho1))
    return np.real(F)**2

fidelities = np.array([fidelity(rho_est[i], y[i]) for i in range(N)])

mean_fidelity = np.mean(fidelities)
std_fidelity  = np.std(fidelities)
print(f"Mean fidelity: {mean_fidelity:.4f} ± {std_fidelity:.4f}")


Mean fidelity: 0.3127 ± 0.1262


# 3) Neural Network Implementation

## 3.1) DNN



In [27]:
X_train, X_test, y_train, y_test = train_test_split(freq_2q, rho_2, train_size=0.7, random_state=2)

In [28]:
def rho_to_alpha(rho):
    # T is lower triangular matrix
    L = np.linalg.cholesky(rho)
    alpha = []
    # extract the reals on the diagonals
    for i in range(rho.shape[0]):
        alpha.append(np.real(L[i, i]))  # add them to alpha
    
    # Off diagonals, contain real and imag components
    for i in range(1, rho.shape[0]):
        for j in range(i):
            alpha.append(np.real(L[i, j]))
            alpha.append(np.imag(L[i, j]))
    return np.array(alpha)

def alpha_to_rho_batch(alpha):
    """Convert batch of alpha vectors to density matrices using Cholesky."""
    N = alpha.shape[0]
    rho = np.zeros((N, 4, 4), dtype=np.complex64)
    for i in range(N):
        a = alpha[i]
        L = np.zeros((4, 4), dtype=np.complex64)
        idx = 0
        for j in range(4):
            L[j, j] = a[idx]
            idx += 1
        for j in range(1, 4):
            for k in range(j):
                re = a[idx]
                im = a[idx + 1]
                L[j, k] = re + 1j * im
                idx += 2
        rho_i = L @ L.conj().T
        rho[i] = rho_i / np.trace(rho_i)
    return rho

# build the full target array
alphas = np.stack([ rho_to_alpha(y_train[i]) for i in range(len(y_train)) ], axis=0)

N_x = X_train.shape[1]   # 36
N_alpha = alphas.shape[1]  # 16

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(N_x,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(N_alpha)   
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="mse",
    metrics=["mse"]
)

history = model.fit(
    X_train, alphas,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
)


Epoch 1/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 866us/step - loss: 0.1568 - mse: 0.1568 - val_loss: 0.0072 - val_mse: 0.0072
Epoch 2/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 814us/step - loss: 0.0163 - mse: 0.0163 - val_loss: 0.0036 - val_mse: 0.0036
Epoch 3/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.0025 - val_mse: 0.0025
Epoch 4/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 769us/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.0022 - val_mse: 0.0022
Epoch 5/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 849us/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0020 - val_mse: 0.0020
Epoch 6/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 849us/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0017 - val_mse: 0.0017
Epoch 7/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 848us/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0017 - val_mse: 0.0017
Epoch 8/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 1s 766us/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.0019 - val_mse: 0.0019
Epoch 9/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 

In [29]:
def fidelity(rho1, rho2):
    """
    Uhlmann fidelity between two density matrices.
    inputs: the two density matrices to be compares
    """
    sqrt_rho1 = sqrtm(rho1)
    product = sqrt_rho1 @ rho2 @ sqrt_rho1
    sqrt_product = sqrtm(product)
    F = np.trace(sqrt_product)
    return np.real(F)**2


alpha_pred = model.predict(X_test)      
rho_pred = alpha_to_rho_batch(alpha_pred)  


# Compute fidelities
fidelities = np.array([
    fidelity(rho_pred[i], y_test[i])
    for i in range(len(rho_pred))
])

# Summary
mean_fid = np.mean(fidelities)
std_fid = np.std(fidelities)
print(f"Test Set Fidelity: {mean_fid:.4f} ± {std_fid:.4f}")


938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 307us/step
Test Set Fidelity: 0.9924 ± 0.0181
